## Start!

In [1]:
def read_json(fname):
    from pathlib import Path
    import json
    assert Path(fname).is_file(), "fked up"
    with open(fname, "rb") as f:
        ret = json.load(f)
        f.close()
    return ret

In [ ]:
!wget -q https://www.dropbox.com/s/3s3io4jb1dy73z6/Question2.json
!wget -q https://www.dropbox.com/s/markixa05xuh75j/Answer2.json

In [2]:
"re-read x"
theque = read_json("Question2.json")
"re-read y"
theans = read_json("Answer2.json")

xdata = theque
ydata = theans

In [3]:
"check if data were parsed"
xdata[:2], ydata[:2]

([[[0.006895913481712338, 0.007801427841186519],
   [0.006895913481712338, 0.007801427841186519],
   [0.0689591348171234, 0.07801427841186521],
   [0.7012757062911987, 0.7854921817779541]],
  [[0.019425401329994194, 0.021897491931915272],
   [0.019425401329994194, 0.021897491931915272],
   [0.13219079196453093, 0.14876206874847409],
   [0.7178986668586731, 0.7966970205307007]]],
 [[[0.018625862314105027, 0.02107165659904479],
   [0.018625862314105027, 0.02107165659904479],
   [0.12419540180563925, 0.14050371541976925]],
  [[0.035710939231753336, 0.04018765605401991],
   [0.035710939231753336, 0.04018765605401991],
   [0.18228078034758566, 0.20479913315296172]]])

### 3 as a group!!!
#### double check the cell right below 

In [4]:
"split the data to 80%-20% first ..."
import random
rr = random.uniform

x_train, y_train, x_test, y_test = [], [], [], []

for i in range(len(xdata)):
  rdx = rr(0,1)
  if rdx > 0.2: # 80% of the time
    x_train.append(xdata[i])
    y_train.append(ydata[i])
  else:
    x_test.append(xdata[i])
    y_test.append(ydata[i])

len(x_train), len(y_train), len(x_test), len(y_test)

(7553, 7553, 1931, 1931)

#### If your machine has tensorboard support, try it

In [5]:
%reload_ext tensorboard

In [7]:
"our pretrained weight, one can load this as a comparison"
#!wget -q https://www.dropbox.com/s/xji5ztmtwbac296/9826model-20220910T034210Z-001.zip
#!unzip 9826model-20220910T034210Z-001.zip
#!ls -l

In [8]:
import tensorflow as tf
import keras

In [9]:
"new_model = keras.models.load_model('9826model')"
"""
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
=================================================================
 flatten_2 (Flatten)         (None, 8)                 0         
                                                                 
 dense_10 (Dense)            (None, 128)               1152      
                                                                 
 dropout_8 (Dropout)         (None, 128)               0         
                                                                 
 dense_11 (Dense)            (None, 64)                8256      
                                                                 
 dropout_9 (Dropout)         (None, 64)                0         
                                                                 
 dense_12 (Dense)            (None, 32)                2080      
                                                                 
 dropout_10 (Dropout)        (None, 32)                0         
                                                                 
 dense_13 (Dense)            (None, 16)                528       
                                                                 
 dropout_11 (Dropout)        (None, 16)                0         
                                                                 
 dense_14 (Dense)            (None, 6)                 102       
                                                                 
 reshape_2 (Reshape)         (None, 3, 2)              0         
                                                                 
=================================================================
Total params: 12,118
Trainable params: 12,118
Non-trainable params: 0
_________________________________________________________________

"""

'\nModel: "sequential_2"\n_________________________________________________________________\n Layer (type)                Output Shape              Param #   \n=================================================================\n flatten_2 (Flatten)         (None, 8)                 0         \n                                                                 \n dense_10 (Dense)            (None, 128)               1152      \n                                                                 \n dropout_8 (Dropout)         (None, 128)               0         \n                                                                 \n dense_11 (Dense)            (None, 64)                8256      \n                                                                 \n dropout_9 (Dropout)         (None, 64)                0         \n                                                                 \n dense_12 (Dense)            (None, 32)                2080      \n                                    

In [10]:
import numpy as np

In [11]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D, Conv1D, Reshape, BatchNormalization, Add

"train our model from scratch"
model_from_scratch = tf.keras.models.Sequential([
    Flatten(input_shape=(4,2,1)),
    Dense(128,activation='relu'),
    Dropout(0.2),
    Dense(64,activation='relu'),
    Dropout(0.2),
    Dense(32,activation='relu'),
    Dropout(0.2),
    Dense(16,activation='relu'),
    Dropout(0.2),
    Dense(6),
    tf.keras.layers.Reshape((3,2), input_shape=(6,))
])

In [12]:
%%time
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

CPU times: total: 0 ns
Wall time: 0 ns


### Remember to change checkpoint path everytime!

In [13]:
checkpoint_path = "the_checkpoint_path/cp-{epoch:04d}.ckpt"

In [14]:
import os

checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 save_freq = 32,
                                                 verbose=1)

In [15]:
def custom_mse(y_true, y_pred):
    
    y_true = tf.cast(y_true,dtype = tf.float32)
    # calculating squared difference between target and predicted values
    loss = 0
    for t in range (0,1):
        loss += 10000*(y_true[0][t]-y_pred[0][t])*(y_true[0][t]-y_pred[0][t])+(y_true[1][t]-y_pred[1][t])*(y_true[1][t]-y_pred[1][t])+(y_true[2][t]-y_pred[2][t])*(y_true[2][t]-y_pred[2][t])
                
    # summing both loss values along batch dimension 
    #loss = K.sum(loss, axis=1)        # (batch_size,)
    
    return loss

In [ ]:
%tensorboard --logdir logs/fit

In [16]:
epochs = 10

### don't be fool by the cell output, this INDEED WORKS!

In [17]:
%%time
model_from_scratch.compile(optimizer='adam',
              loss=custom_mse,
              #metrics=[tf.keras.metrics.MeanSquaredError()])
              metrics=['Accuracy'])
"To use tensorboard, don't assign a variable to store return value from 'model.fit()'"
"aka, don't say 'history = model.fit(...)'"
model_from_scratch.fit(x=x_train, y=y_train, validation_data=(x_test, y_test), batch_size = 32, epochs=epochs, callbacks=[cp_callback, tensorboard_callback])

Epoch 1/10
 31/237 [==>...........................] - ETA: 0s - loss: 18.4296 - Accuracy: 0.4805 
Epoch 1: saving model to training_segtext_in_tfopt_env_6th_attmp\cp-0001.ckpt
 57/237 [======>.......................] - ETA: 1s - loss: 12.5585 - Accuracy: 0.4454
Epoch 1: saving model to training_segtext_in_tfopt_env_6th_attmp\cp-0001.ckpt
 93/237 [==========>...................] - ETA: 0s - loss: 8.6488 - Accuracy: 0.4140
Epoch 1: saving model to training_segtext_in_tfopt_env_6th_attmp\cp-0001.ckpt
115/237 [=============>................] - ETA: 0s - loss: 7.4238 - Accuracy: 0.4102
Epoch 1: saving model to training_segtext_in_tfopt_env_6th_attmp\cp-0001.ckpt
150/237 [=================>............] - ETA: 0s - loss: 6.1121 - Accuracy: 0.3886
Epoch 1: saving model to training_segtext_in_tfopt_env_6th_attmp\cp-0001.ckpt
182/237 [======================>.......] - ETA: 0s - loss: 5.2462 - Accuracy: 0.3897
Epoch 1: saving model to training_segtext_in_tfopt_env_6th_attmp\cp-0001.ckpt
221/237 

InvalidArgumentError: Graph execution error:

Detected at node 'custom_mse/strided_slice_16' defined at (most recent call last):
    File "c:\Users\Kelvin\anaconda3\envs\tf_model_opt\lib\runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\Kelvin\anaconda3\envs\tf_model_opt\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\Kelvin\anaconda3\envs\tf_model_opt\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "c:\Users\Kelvin\anaconda3\envs\tf_model_opt\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "c:\Users\Kelvin\anaconda3\envs\tf_model_opt\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "c:\Users\Kelvin\anaconda3\envs\tf_model_opt\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forev

In [24]:
# Control TensorBoard display. If no port is provided, 
# the most recently launched TensorBoard is used
from tensorboard import notebook
notebook.list() # View open TensorBoard instances

Known TensorBoard instances:
  - port 6006: logdir logs (started 3:05:42 ago; pid 17456)
  - port 6006: logdir logs/fit (started 2:18:46 ago; pid 6028)


In [22]:
notebook.display(port=6006, height=1000)